# Experience Replay

In this project we study the influence of experience replay to improve Q-Learning.
We study the following experience replay techniques,
- Uniform ER
- Prioritized ER
- Hindsight ER
- Combined ER

Our experiements are limited to a subset of the Open-AI Gym's Classic Control environments,
- Cart Pole
- Mountain Car
- Acrobot

Using these ER methods for Q-Learning in the Gym environments, we measure the influence of i.i.d. and TD-error to quantiatively compare the performances.

Our hypothesis is that the effect of i.i.d. has a greater influence in the performance in comparision to the TD-error.

### Setup

In this section we load the required libraries to implement Q-Learning and run the environments of Open-AI Gym.

In [ ]:
%matplotlib inline
import os
import sys

import pickle
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict

nested_dict = lambda: defaultdict(nested_dict)

import torch
from torch import nn, optim
import torch.nn.functional as F

import gym
gym.logger.set_level(40)

from tqdm import tqdm as _tqdm

def tqdm(*args, **kwargs):
    return _tqdm(*args, **kwargs, mininterval=1)

assert sys.version_info[:3] >= (3, 6, 0), "Make sure you have Python 3.6 installed!"

## Q-Learning

Here, we define a Neural Network for the Q function. The forward pass accepts the observation state and outputs the action values.

In [ ]:
class QNetwork(nn.Module):
    def __init__(self, input_dim=4, num_hidden=128, output_dim=2):
        nn.Module.__init__(self)
        self.l1 = nn.Linear(input_dim, num_hidden)
        self.l2 = nn.Linear(num_hidden, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.l2(self.relu(self.l1(x)))

In [ ]:
# Let's instantiate and test if it works
input_dim = 4
num_hidden = 128
output_dim = 2
torch.manual_seed(1234)
model = QNetwork(input_dim, num_hidden, output_dim)

torch.manual_seed(1234)
test_model = nn.Sequential(
    nn.Linear(input_dim, num_hidden), 
    nn.ReLU(), 
    nn.Linear(num_hidden, output_dim)
)

x = torch.rand(10, 4)

# at this point we do not need backpropagation
with torch.no_grad():
    assert np.allclose(model(x).numpy(), test_model(x).numpy())

Some useful functions for training,

In [ ]:
def get_epsilon(it):
    if it == 0:
        return 1
    return np.maximum(1 - it/1000, 0.05)

def select_action(model, state, epsilon):
    with torch.no_grad():
        actionProbs = model(torch.Tensor(state)).numpy()
    action = np.random.choice(len(actionProbs), 1) if np.random.rand() < epsilon else np.argmax(actionProbs)
    return int(action.squeeze()), actionProbs[0][action]

def copy_model(source_model, target_model):
    target_model.load_state_dict(source_model.state_dict())

def append_sample(transition, model, target_model, discount_factor, memory):
    state, action, reward, next_state, done = transition
    target = model(torch.FloatTensor(state)).data
    old_val = target[0][action].item()
    target_val = target_model(torch.FloatTensor(next_state)).data
    if done:
        target[0][action] = reward
    else:
        target[0][action] = reward + discount_factor * torch.max(target_val)
    new_val = target[0][action].item()
    error = abs(old_val - new_val)
    transition = (state, action, reward, next_state, done)
    memory.push(transition, error)
    
    
def correlation_count(state, threshold=0.8):
    cov=np.cov(state)
    corrcoeff = cov.copy()
    n_samples = cov.shape[0]
    for i in range(n_samples):
        corrcoeff[i,:]=corrcoeff[i,:]/np.sqrt(cov[i,i])
    for j in range(n_samples):
        corrcoeff[:,j]=corrcoeff[:,j]/np.sqrt(cov[j,j])
    
    return (np.sum(np.abs(corrcoeff)>threshold) - n_samples) / 2

In [ ]:
def train(memory, model, target_model, batch_size, action_size, optimizer, discount_factor):

    if len(memory) < batch_size:
        return None, None, None #To match length of output

    transition_and_error, idxs, is_weights = memory.sample(batch_size)
    
    #Separate mini-batch into transition and TD error
    mini_batch, sampled_TD_error = zip(*transition_and_error)    
    sampled_TD_error = np.asarray(sampled_TD_error)
    
    #Set up data for training
    mini_batch = np.array(mini_batch).transpose()
    states = np.vstack(mini_batch[0])
    actions = list(mini_batch[1])
    rewards = list(mini_batch[2])
    next_states = np.vstack(mini_batch[3])
    dones = mini_batch[4]

    #Collect stats
    error_stats = (sampled_TD_error.mean(), np.median(sampled_TD_error), sampled_TD_error.max(), sampled_TD_error.std())
    iid_stats = correlation_count(states)
    
    # bool to binary
    dones = dones.astype(int)

    # Q function of current state
    states = torch.Tensor(states)
    pred = model(states)

    # one-hot encoding
    a = torch.LongTensor(actions).view(-1, 1)

    one_hot_action = torch.zeros(batch_size, action_size)
    one_hot_action.scatter_(1, a, 1)

    pred = torch.sum(pred.mul(torch.tensor(one_hot_action)), dim = 1)

    # Q function of next state
    next_states = torch.Tensor(next_states)
    next_pred = target_model(next_states).data

    rewards = torch.FloatTensor(rewards)
    dones = torch.FloatTensor(dones)

    # Q Learning: get maximum Q value at s' from target model
    target = rewards + (1 - dones) * discount_factor * next_pred.max(1)[0]
    target = torch.autograd.Variable(target)

    errors = torch.abs(pred - target).data.numpy()
    
    # update priority
    if isinstance(memory, PrioritizedMemory):
        for i in range(batch_size):
            idx = idxs[i]
            memory.update(idx, errors[i])

    optimizer.zero_grad()

    # MSE Loss function
    loss = F.mse_loss(pred, target)
    loss.backward()

    # and train
    optimizer.step()
    
    return loss.item(), error_stats, iid_stats

In [ ]:
def run_episodes(model, target_model, memory, env, num_episodes, batch_size, discount_factor, learn_rate):
    optimizer = optim.Adam(model.parameters(), learn_rate)
    
    #initialize output
    results_output = {}
    results_output['durations'] = np.zeros(num_episodes)
    results_output['mean_TD_error'] = np.zeros(num_episodes)
    results_output['median_TD_error'] = np.zeros(num_episodes)
    results_output['max_TD_error'] = np.zeros(num_episodes)
    results_output['std_TD_error'] = np.zeros(num_episodes)
    results_output['correlation_count'] = np.zeros(num_episodes)
    results_output['Q_values'] = []
    results_output['rewards'] = []
    
    iteration = 0
    
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n

    scores = []

    for episode in tqdm(range(num_episodes)):
        done = False
        score = 0
        
        mean_TD_error = []
        median_TD_error = []
        max_TD_error = []
        std_TD_error = []
        correlation_count = []
        Q_values = []
        rewards = []
        
        state = env.reset()
        state = np.reshape(state, [1, state_size])

        while not done:
            action, Q_val = select_action(model, state, get_epsilon(iteration))
            
            next_state, reward, done, info = env.step(action)
            next_state = np.reshape(next_state, [1, state_size])
            Q_values.append(Q_val)
            
            # if an action make the episode end, then gives penalty of -10
            reward = reward if not done or score == 499 else -10
            rewards.append(reward)

            transition = (state, action, reward, next_state, done)
    
            append_sample(transition, model, target_model, discount_factor, memory)
    
            # Train
            loss, error_stats, iid_stats = train(memory, model, target_model, batch_size, action_size, optimizer, discount_factor)
            
            # Update
            score += reward
            state = next_state
            iteration += 1
            
            # Append stats
            if error_stats != None:
                results_output['durations'][episode] += 1
                mean_TD_error.append(error_stats[0])
                median_TD_error.append(error_stats[1])
                max_TD_error.append(error_stats[2])
                std_TD_error.append(error_stats[3])
                correlation_count.append(iid_stats)

            if done:
                results_output['Q_values'].append(np.array([x for x in Q_values[:]]))
                results_output['rewards'].append(np.array([x for x in rewards[:]]))
                # every episode update the target model to be same with model
                copy_model(model, target_model)

                # every episode, plot the play time
                score = score if score == 500 else score + 10
                scores.append(score)
                 
        #Average stats per episode
        if len(mean_TD_error) > 0:
            results_output['mean_TD_error'][episode] = np.array(mean_TD_error).mean()
            results_output['median_TD_error'][episode] = np.array(median_TD_error).mean()
            results_output['max_TD_error'][episode] = np.array(max_TD_error).mean()
            results_output['std_TD_error'][episode] = np.array(std_TD_error).mean()
            results_output['correlation_count'][episode] = np.array(correlation_count).mean()
                
    return results_output

## Environment

We load the necessary Gym environments required for our experiments.

In [ ]:
environments = ["CartPole-v0", "Acrobot-v1", "MountainCar-v0"]
envs = {}

print('Environment \t\t State Space \t Action Space')
print('----------- \t\t ----------- \t ------------')
for environment in environments:
    env = gym.envs.make(environment)
    envs[environment] = env
    print('{} \t\t {} \t {}'.format(
        environment,
        env.observation_space,
        env.action_space)
    )

## Experience Replay

### ReplayMemory

In [ ]:
class ReplayMemory:
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []

    def push(self, transition, error):
        self.memory.append((transition, error))

    def sample(self, batch_size):
        raise NotImplementedError('Function sample is not defined')

    def __len__(self):
        return len(self.memory)

### No Experience Replay and Uniform Experience Replay

In [ ]:
class NoneMemory(ReplayMemory):
    def sample(self, batch_size):
        #Samples a tuple of tuple of transitions and error - ((s, a, r, s', done), TDerror)
        return self.memory[-batch_size:], None, None # extra Nones to have the same behaviour as PER


class UniformMemory(ReplayMemory):
    def sample(self, batch_size):
        #Samples a tuple of tuple of transitions and error - ((s, a, r, s', done), TDerror)
        index = np.random.choice(len(self), batch_size, replace=False)
        return [self.memory[i] for i in index], None, None # extra Nones to have the same behaviour as PER


class CombinedMemory(ReplayMemory):
    def sample(self, batch_size):
        #Samples a tuple of tuple of transitions and error - ((s, a, r, s', done), TDerror)
        index = np.random.choice(len(self) - 1, batch_size - 1, replace=False)
        samples = [self.memory[i] for i in index]
        samples.append(self.memory[-1])
        return samples, None, None # extra Nones to have the same behaviour as PER

### Prioritized Experience Replay

In [ ]:
# Source: https://github.com/rlcode/per

# SumTree
# a binary tree data structure where the parent’s value is the sum of its children
class SumTree:
    write = 0

    def __init__(self, capacity):
        self.capacity = capacity
        self.tree = np.zeros(2 * capacity - 1)
        self.data = np.zeros(capacity, dtype = object)
        self.n_entries = 0

    # update to the root node
    def _propagate(self, idx, change):
        parent = (idx - 1) // 2

        self.tree[parent] += change

        if parent != 0:
            self._propagate(parent, change)

    # find sample on leaf node
    def _retrieve(self, idx, s):
        left = 2 * idx + 1
        right = left + 1

        if left >= len(self.tree):
            return idx

        if s <= self.tree[left]:
            return self._retrieve(left, s)
        else:
            return self._retrieve(right, s - self.tree[left])

    def total(self):
        return self.tree[0]

    # store priority and sample
    def add(self, p, data):
        idx = self.write + self.capacity - 1

        self.data[self.write] = data
        self.update(idx, p)

        self.write += 1
        if self.write >= self.capacity:
            self.write = 0

        if self.n_entries < self.capacity:
            self.n_entries += 1

    # update priority
    def update(self, idx, p):
        change = p - self.tree[idx]

        self.tree[idx] = p
        self._propagate(idx, change)

    # get priority and sample
    def get(self, s):
        idx = self._retrieve(0, s)
        dataIdx = idx - self.capacity + 1

        return (idx, self.tree[idx], self.data[dataIdx])


class PrioritizedMemory:  # stored as ( s, a, r, s_ ) in SumTree
    e = 0.01
    a = 0.6
    beta = 0.4
    beta_increment_per_sampling = 0.001

    def __init__(self, capacity):
        self.tree = SumTree(capacity)
        self.capacity = capacity

    def _get_priority(self, error):
        return (error + self.e) ** self.a
    
    def __len__(self):
        return self.tree.n_entries

    def push(self, sample, error):
        p = self._get_priority(error)
        self.tree.add(p, (sample, error))

    def sample(self, n):
        batch = []
        idxs = []
        segment = self.tree.total() / n
        priorities = []

        self.beta = np.min([1., self.beta + self.beta_increment_per_sampling])

        for i in range(n):
            a = segment * i
            b = segment * (i + 1)

            s = random.uniform(a, b)
            (idx, p, data) = self.tree.get(s)
            priorities.append(p)
            batch.append(data)
            idxs.append(idx)

        sampling_probabilities = priorities / self.tree.total()
        is_weight = np.power(self.tree.n_entries * sampling_probabilities, -self.beta)
        is_weight /= is_weight.max()
        
        return batch, idxs, is_weight

    def update(self, idx, error):
        p = self._get_priority(error)
        self.tree.update(idx, p)

Test if the memory module works as expected,

In [ ]:
capacity = 10
memory = NoneMemory(capacity)

# Sample a transition
s = env.reset()
a = env.action_space.sample()
s_next, r, done, _ = env.step(a)

# Push a transition
memory.push((s, a, r, s_next, done), 0)

# Sample a batch size of 1
print(memory.sample(1)[0][0])

Test if the train module works as expected,

In [ ]:
# settings
batch_size = 64
discount_factor = 0.8
learn_rate = 1e-3
input_dim = 2
output_dim = 2

model = QNetwork(input_dim, num_hidden, output_dim)
optimizer = optim.Adam(model.parameters(), learn_rate)

# setup memory
transition = memory.sample(1)[0][0][0]
memory = UniformMemory(10 * batch_size)

# fill with dummy data
for i in range(batch_size):
    memory.push(transition, 0)

# must prints loss, meanTDerror, (medianTDerror, maxTDerror, stdTDerror), Correlation count
print(train(memory, model, model, batch_size, output_dim, optimizer, discount_factor))

# Run Experiments

In [ ]:
def run(er_type='uniform', environment="CartPole-v0"):

    # load environment
    env = envs[environment]
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n
    num_hidden = 128

    # init networks
    model = QNetwork(state_size, num_hidden, action_size)
    target_model = QNetwork(state_size, num_hidden, action_size)

    copy_model(model, target_model)

    # run settings
    batch_size = 64
    discount_factor = 0.8
    learn_rate = 1e-3
    memory_size = 10000
    num_episodes = 300
    
    er_types = {
        'none': NoneMemory,
        'uniform': UniformMemory,
        'prioritized': PrioritizedMemory,
        'combined': CombinedMemory
    }

    memory = er_types[er_type](memory_size)

    #Return a dictionary of the relevant statistics per episode
    metrics = run_episodes(model, target_model, memory, env, num_episodes, batch_size, discount_factor, learn_rate)
    
    return metrics

## Results

In [ ]:
exp_replays = ['none', 'uniform', 'prioritized', 'combined']
environments = ['CartPole-v0', 'Acrobot-v1', 'MountainCar-v0']

ITERATIONS = 50

def grid_search(iterations):
    results = nested_dict()
    for environment in environments:
        for er_type in exp_replays:
            for iteration in range(iterations):
                print('Running {} in {}'.format(er_type, environment))
                results[environment][er_type][iteration] = run(er_type, environment)
    return results

def save(data):
    with open('results.picke', 'wb') as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)
# use results and then next cell for plotting - no need to get results multiple times
results = grid_search(100)
# results = run('prioritized', 'Acrobot-v1')
save(results)

In [ ]:
results.keys()

In [ ]:
smooth_range = 10

# And see the results
def smooth(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / float(N)

def plot_Q_values(Q_values, spacing):
    for i in range(0, len(Q_values), spacing):
        plt.subplot(len(Q_values)/spacing,1, i/spacing+1)
        plt.plot(Q_values[i])
        plt.title('Q_values in episode {} '.format(i))
    # plt.subplots_adjust(top=1, bottom=-1.5)
    plt.show()

def grid_graphs():
    # plot durations
    # compare per environment
    for index, env in enumerate(environments):
        plt.subplot(len(environments),1, index+1)
        for er_type in exp_replays:
            plt.plot(smooth(results[env][er_type]['durations'], smooth_range))
        plt.title('Episode durations per episode in {} environment'.format(env))
        plt.legend(exp_replays)
    # plt.subplots_adjust(top=1, bottom=-1.5)
    plt.show()
    
    # compare per ER
    for index, er_type in enumerate(exp_replays):
        plt.subplot(len(exp_replays),1, index+1)
        for env in environments:
            plt.plot(smooth(results[env][er_type]['durations'], smooth_range))
        plt.title('Episode durations per episode with {} experience replay'.format(exp_replays[index]))
        plt.legend(environments)
    plt.subplots_adjust(top=1, bottom=-1.5)
    plt.show()
    
# grid_graphs()
# print(results["CartPole-v0"])
plot_Q_values(results['Q_values'], 25)